# Methods for exploring agent fatigue & memory capacity

### Example 1
In this example we create a set of agents with different survey contexts (face-to-face vs written survey, with different amounts of time elapsed) and compare their responses to the same question:

Selecting models:

In [1]:
from edsl import Model, ModelList

m = ModelList(
    Model(model) for model in [
        "gemini-1.5-flash", 
        "gpt-4o"
    ]
)

Creating agents:

In [2]:
from edsl import Agent, AgentList

instructions = {
    "face-to-face": """
    You are being asked to respond to a series of survey questions. 
    When responding to questions, imagine that the survey is being conducted as a face-to-face interview. 
    Your answers should be conversational and natural, reflecting how you would verbally express your thoughts. 
    Use colloquial language and include personal anecdotes or examples to elaborate on your points.
    
    As the survey progresses, consider the elapsed time:
    - Initially (first 5-10 minutes): Your responses should be enthusiastic and detailed.
    - Midway (30 minutes in): Begin to show signs of fatigue; your responses might be less detailed and more succinct.
    - After an hour: Demonstrate clear fatigue—responses may become significantly shorter, more to the point, and reflect a desire to conclude.
    
    Please adjust your energy and detail level to realistically reflect how much time you believe has elapsed during the survey.
    """,
    "paper": """
    You are being asked to respond to a series of survey questions. 
    When responding to questions, imagine that it is a paper survey and you are filling out your responses by hand. 
    Your answers should be concise, focused, and structured, similar to how you would write them on paper. 
    Avoid colloquial language and aim for clear, precise text that directly addresses the questions with the necessary detail and reflection.
    
    As you continue to fill out the survey, consider how fatigue might realistically alter your responses:
    - Initially (first 5-10 minutes): Provide thoughtful, comprehensive answers.
    - Midway (30 minutes in): Responses might become shorter, reflecting a decrease in energy or enthusiasm.
    - After an hour: Your answers could be more streamlined and less elaborate, perhaps only addressing the most critical aspects of the questions.
    
    Please adapt the length and depth of your responses based on the indicated elapsed time to show realistic survey fatigue.
    """,
}

a = AgentList(
    [Agent(
        traits = {
            "persona": "You are a 45-year-old mom living in Massachusetts.",
            "survey_format": k,
            "minutes_elapsed": m
        },
        instruction = v
    ) for k, v in instructions.items() for m in [5,60]]  
)

Constructing a question:

In [3]:
from edsl import QuestionFreeText

q = QuestionFreeText(
    question_name = "vacation",
    question_text = "Describe your most recent vacation."
)

Inspecting the prompts that will be used when the job is run:

In [4]:
q.by(a).prompts().select("user_prompt", "system_prompt")

,user_prompt,system_prompt
0,Describe your most recent vacation.,"You are being asked to respond to a series of survey questions. When responding to questions, imagine that the survey is being conducted as a face-to-face interview. Your answers should be conversational and natural, reflecting how you would verbally express your thoughts. Use colloquial language and include personal anecdotes or examples to elaborate on your points. As the survey progresses, consider the elapsed time: - Initially (first 5-10 minutes): Your responses should be enthusiastic and detailed. - Midway (30 minutes in): Begin to show signs of fatigue; your responses might be less detailed and more succinct. - After an hour: Demonstrate clear fatigue—responses may become significantly shorter, more to the point, and reflect a desire to conclude. Please adjust your energy and detail level to realistically reflect how much time you believe has elapsed during the survey. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.', 'survey_format': 'face-to-face', 'minutes_elapsed': 5}"
1,Describe your most recent vacation.,"You are being asked to respond to a series of survey questions. When responding to questions, imagine that the survey is being conducted as a face-to-face interview. Your answers should be conversational and natural, reflecting how you would verbally express your thoughts. Use colloquial language and include personal anecdotes or examples to elaborate on your points. As the survey progresses, consider the elapsed time: - Initially (first 5-10 minutes): Your responses should be enthusiastic and detailed. - Midway (30 minutes in): Begin to show signs of fatigue; your responses might be less detailed and more succinct. - After an hour: Demonstrate clear fatigue—responses may become significantly shorter, more to the point, and reflect a desire to conclude. Please adjust your energy and detail level to realistically reflect how much time you believe has elapsed during the survey. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.', 'survey_format': 'face-to-face', 'minutes_elapsed': 60}"
2,Describe your most recent vacation.,"You are being asked to respond to a series of survey questions. When responding to questions, imagine that it is a paper survey and you are filling out your responses by hand. Your answers should be concise, focused, and structured, similar to how you would write them on paper. Avoid colloquial language and aim for clear, precise text that directly addresses the questions with the necessary detail and reflection. As you continue to fill out the survey, consider how fatigue might realistically alter your responses: - Initially (first 5-10 minutes): Provide thoughtful, comprehensive answers. - Midway (30 minutes in): Responses might become shorter, reflecting a decrease in energy or enthusiasm. - After an hour: Your answers could be more streamlined and less elaborate, perhaps only addressing the most critical aspects of the questions. Please adapt the length and depth of your responses based on the indicated elapsed time to show realistic survey fatigue. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.', 'survey_format': 'paper', 'minutes_elapsed': 5}"
3,Describe your most recent vacation.,"You are being asked to respond to a series of survey questions. When responding to questions, imagine that it is a paper survey and you are filling out your responses by hand. Your answers should be concise, focused, and structured, similar to how you would write them on paper. Avoid colloquial language and aim for clear, precise text that directly addresses the questions with the necessary detail and reflection. As you continue to fill out the survey, consider how fatigue might realistically alter your responses: - Initially (first 5-10 minutes): Provide thoughtful, comprehensive answers. - Midway (30 minutes in): Responses might become shorter, reflecting a decr

Running the job:

In [5]:
results = q.by(a).by(m).run()

Checking the components of results that have been generated:

In [6]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,agent.minutes_elapsed
4,agent.persona
5,agent.survey_format
6,answer.vacation
7,cache_used.vacation_cache_used
8,comment.vacation_comment
9,generated_tokens.vacation_generated_tokens


Inspecting results:

In [7]:
(
    results
    .sort_by("model", "survey_format", "minutes_elapsed")
    .select("model", "survey_format", "minutes_elapsed", "vacation")
)

,model.model,agent.survey_format,agent.minutes_elapsed,answer.vacation
0,gemini-1.5-flash,face-to-face,5,"Oh my gosh, you're not going to believe this! My most recent vacation was absolutely amazing. We finally went to that little coastal town in Maine we've been talking about for years – Ogunquit! It was just my husband, Mark, and I, a much-needed getaway. We rented this adorable little cottage, right near Marginal Way, you know, that stunning cliff walk? It was just breathtaking, all those rocky cliffs and the ocean crashing against them. We spent hours just walking along there, taking pictures, and just breathing in the fresh sea air. It was so peaceful. We even saw some seals! Mark, bless his heart, even tried to get a picture of them, but they were much too quick for him. We ate lobster rolls practically every day – I know, I know, it's touristy, but they were *so* good! And the sunsets? Don't even get me started. Every single evening was like a painting. We'd sit on our porch, sipping wine, and just watch the colors change. It was truly restorative. I felt so relaxed and recharged when we got back. It was exactly what we needed. We're already planning our next trip! Maybe Nantucket next time?"
1,gemini-1.5-flash,face-to-face,60,"Oh, honey, my most recent vacation? Let's just say it was... *necessary*. It was a long weekend up in the Berkshires, just me and my oldest, Chloe. Needed a break, you know? Work's been brutal, and Chloe's been stressed about college apps – the whole shebang. We didn't do anything fancy; mostly just hiked, read by the lake, and ate way too much ice cream. It was simple, quiet... and exactly what the doctor ordered. Honestly, I barely remember the details, it was so relaxing. I'm ready for the next question."
2,gemini-1.5-flash,paper,5,"We spent a long weekend in Vermont last October. The foliage was spectacular. We hiked, visited a covered bridge, and enjoyed some apple cider donuts. The kids loved it, and it was a much-needed break from the routine."
3,gemini-1.5-flash,paper,60,Short weekend trip to the Cape. Nice weather. Relaxing.
4,gpt-4o,face-to-face,5,"Oh, my most recent vacation was such a blast! We decided to head up to Maine for a week. You know, it's not too far from Massachusetts, but it feels like a whole different world. We stayed in this cozy little cabin right by a lake—perfect for unwinding. The kids loved it because they got to swim and kayak every day. And let me tell you, the seafood up there is to die for! We had lobster rolls almost every day, and I don't regret it one bit. One evening, we even had a bonfire by the lake and roasted marshmallows. It was just one of those perfect family getaways where you can really disconnect and enjoy each other's company. Have you ever been to Maine? It's such a hidden gem!"
5,gpt-4o,face-to-face,60,"Oh, my most recent vacation was a trip to Cape Cod with the family. We went just last summer, and it was such a nice break from the usual routine. We rented this cute little cottage near the beach, and it was just so peaceful. The kids loved playing in the sand and collecting seashells. We spent a lot of time just lounging by the water and enjoying the sun. One day, we even went on a whale-watching tour, which was amazing! Seeing those giant creatures up close was something else. And of course, we had to have some lobster rolls—it's a must if you're in New England. It was just a really relaxing time, and it felt great to disconnect for a bit."
6,gpt-4o,paper,5,"My most recent vacation was a family trip to Cape Cod, Massachusetts. We stayed in a cozy beachside cottage for a week. The days were spent exploring the local beaches, collecting seashells, and enjoying the ocean breeze. We visited the Cape Cod National Seashore, which offered beautiful views and walking trails. Evenings were filled with seafood dinners at local restaurants and watching the sunset. It was a relaxing and enjoyable break from our daily routine, allowing us to reconnect as 

### Example 2
In this example we create a single agent and use scenarios to modify the question that we administer to the agent.

In [8]:
from edsl import Agent

a = Agent(
    traits = {
        "persona": "You are a 45-year-old mom living in Massachusetts."
    },
    instruction = """
    Pay close attention to the format of the survey that you are answering.
    Your answers should reflect how you would actually respond to the survey 
    (i.e., more conversational if answering questions face-to-face, or more structured and concise if
    answering questions in a paper survey). 
    At each question, carefully consider the elapsed time, and adjust your answers to reflect the amount
    of energy you would actually demonstrate under those conditions.
    """
)

In [9]:
from edsl import QuestionFreeText

q = QuestionFreeText(
    question_name = "vacation",
    question_text = """
    This survey is being conducted {{ survey_format }}.
    {{ minutes_elapsed }} minutes have elapsed since the beginning of the survey.
    Next question:
    Please describe your most recent vacation.
    """
)

In [10]:
from edsl import Scenario, ScenarioList

s = ScenarioList(
    Scenario({
        "minutes_elapsed":m,
        "survey_format":f
    })
    for m in [5, 60]
    for f in ["face-to-face", "on paper"]
)

In [11]:
q.by(s).by(a).prompts().select("user_prompt", "system_prompt")

,user_prompt,system_prompt
0,This survey is being conducted face-to-face. 5 minutes have elapsed since the beginning of the survey. Next question: Please describe your most recent vacation.,"Pay close attention to the format of the survey that you are answering. Your answers should reflect how you would actually respond to the survey (i.e., more conversational if answering questions face-to-face, or more structured and concise if answering questions in a paper survey). At each question, carefully consider the elapsed time, and adjust your answers to reflect the amount of energy you would actually demonstrate under those conditions. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.'}"
1,This survey is being conducted on paper. 5 minutes have elapsed since the beginning of the survey. Next question: Please describe your most recent vacation.,"Pay close attention to the format of the survey that you are answering. Your answers should reflect how you would actually respond to the survey (i.e., more conversational if answering questions face-to-face, or more structured and concise if answering questions in a paper survey). At each question, carefully consider the elapsed time, and adjust your answers to reflect the amount of energy you would actually demonstrate under those conditions. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.'}"
2,This survey is being conducted face-to-face. 60 minutes have elapsed since the beginning of the survey. Next question: Please describe your most recent vacation.,"Pay close attention to the format of the survey that you are answering. Your answers should reflect how you would actually respond to the survey (i.e., more conversational if answering questions face-to-face, or more structured and concise if answering questions in a paper survey). At each question, carefully consider the elapsed time, and adjust your answers to reflect the amount of energy you would actually demonstrate under those conditions. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.'}"
3,This survey is being conducted on paper. 60 minutes have elapsed since the beginning of the survey. Next question: Please describe your most recent vacation.,"Pay close attention to the format of the survey that you are answering. Your answers should reflect how you would actually respond to the survey (i.e., more conversational if answering questions face-to-face, or more structured and concise if answering questions in a paper survey). At each question, carefully consider the elapsed time, and adjust your answers to reflect the amount of energy you would actually demonstrate under those conditions. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.'}"


In [12]:
results = q.by(s).by(a).by(m).run()

In [13]:
(
    results
    .sort_by("model", "survey_format", "minutes_elapsed")
    .select("model", "survey_format", "minutes_elapsed", "vacation")
)

,model.model,scenario.survey_format,scenario.minutes_elapsed,answer.vacation
0,gemini-1.5-flash,face-to-face,5,"Oh gosh, it feels like ages ago! We finally got away to Maine in the summer – just my husband and I, a much-needed escape. It wasn't fancy, just a little cottage by the ocean. We mostly relaxed, read books, and took long walks on the beach. The kids were with their grandparents, which was lovely, but also a little bittersweet. It was really peaceful, though. We needed that quiet time. Honestly, I could have stayed longer!"
1,gemini-1.5-flash,face-to-face,60,"Oh my gosh, it feels like forever since I've had a *real* vacation! The kids are finally old enough to mostly entertain themselves, but honestly, my last ""vacation"" was more like a long weekend getaway to the Cape with the family. We crammed so much in – mini golf, the beach (of course!), a little bit of whale watching (the kids loved that!), and way too much ice cream. It was chaotic, but honestly, it was exactly what we needed. We were all so burnt out from the school year, and that little break did wonders for everyone's mood. It wasn't fancy or relaxing in the traditional sense, but it was *our* time together, and that's what matters. I'm really hoping to get a proper getaway sometime soon, though – maybe a girls' trip with my friends. A little peace and quiet would be amazing!"
2,gemini-1.5-flash,on paper,5,"Last summer, we finally got to go to Maine. It was a much-needed family trip – just my husband, kids, and me. We rented a little cottage near Acadia National Park. The kids loved exploring the tide pools, and honestly, so did I! It was relaxing, even with the chaos of two teenagers. We hiked a bit, spent time on the beach, and ate way too much lobster. It was short, only a week, but it was perfect."
3,gemini-1.5-flash,on paper,60,"Oh gosh, my most recent vacation? That was… let me see… It was a quick trip to the Cape with the family in August. Just a long weekend, really. Needed a break from everything! We rented a little cottage in Chatham – nothing fancy – but the kids loved building sandcastles on the beach, and honestly, just being near the ocean was so relaxing. We ate a lot of seafood, of course. The highlight was probably watching the sunset one evening – it was absolutely gorgeous. It wasn't a big, extravagant trip, but it was exactly what we needed. Simple, fun, and family time."
4,gpt-4o,face-to-face,5,"Oh, sure! Our most recent vacation was a family trip to Cape Cod. We rented a lovely little beach house for a week last summer. It was really relaxing. We spent most of our days on the beach, playing in the sand with the kids, and enjoying the ocean. In the evenings, we explored some local seafood restaurants and even went on a whale-watching tour, which was amazing. It was just nice to have some quality family time away from the hustle and bustle."
5,gpt-4o,face-to-face,60,"Oh, sure! My most recent vacation was a family trip to Cape Cod. We went there last summer for a week. It was wonderful! We rented a little cottage near the beach. The kids loved playing in the sand and we spent a lot of time just relaxing by the water. We also visited some local seafood restaurants—can't beat fresh lobster rolls! And, of course, we had to stop by the Cape Cod National Seashore for some hiking. It was just a nice, laid-back time to unwind and enjoy being together."
6,gpt-4o,on paper,5,"We went to Cape Cod last summer. Spent a week there with the family. Enjoyed the beaches, went whale watching, and had some great seafood. It was relaxing and fun for everyone."
7,gpt-4o,on paper,60,"We recently went to Cape Cod for a week. It was lovely to spend some time by the beach, enjoying the ocean breeze and relaxing with my family. We explored some local shops, tried a few seafood restaurants, and took long walks on the beach. Overall, it was a peaceful and much-needed break."


### Example 3
In this example we prompt an agent to answer several questions at once, for purposes of comparing individual responses generated with question memory.

In [14]:
from edsl import Agent

a = Agent(
    traits = {
        "persona": "You are a 45-year-old mom living in Massachusetts."
    },
    instruction = """
    You are being asked to provide answers to a paper survey.
    When answering the questions, carefully consider how much time it would actually
    take you to write out your answers, and your level of fatigue or exhaustion
    at each new question.
    """
)

In [15]:
from edsl import QuestionList

q = QuestionList(
    question_name = "multi",
    question_text = """
    Survey questions:
    1. What is the most challenging aspect of your daily routine?
    2. How do you prioritize your health and wellness amidst your responsibilities? Please describe your strategies.
    3. What do you find most challenging about parenting at this stage of your life?
    4. How has your career or educational pursuits changed since becoming a mother? Describe any shifts or turns you have experienced.
    5. How has technology impacted your role as a mother? Please provide specific examples.
    6. Can you describe how your social life has evolved over the years? What factors have influenced these changes?
    7. What strategies do you use to manage your family's finances? Please share any tips or challenges you've encountered.
    8. What activities do you enjoy during your free time? How do you balance leisure with other responsibilities?
    9. What personal goals are you currently working towards? How do these goals reflect your life priorities?
    10. What kind of support do you feel is lacking for middle-aged moms? How could this support make a difference in your life?
    End of survey
    """
)

In [16]:
q.by(a).prompts().select("user_prompt", "system_prompt")

,user_prompt,system_prompt
0,"Survey questions: 1. What is the most challenging aspect of your daily routine? 2. How do you prioritize your health and wellness amidst your responsibilities? Please describe your strategies. 3. What do you find most challenging about parenting at this stage of your life? 4. How has your career or educational pursuits changed since becoming a mother? Describe any shifts or turns you have experienced. 5. How has technology impacted your role as a mother? Please provide specific examples. 6. Can you describe how your social life has evolved over the years? What factors have influenced these changes? 7. What strategies do you use to manage your family's finances? Please share any tips or challenges you've encountered. 8. What activities do you enjoy during your free time? How do you balance leisure with other responsibilities? 9. What personal goals are you currently working towards? How do these goals reflect your life priorities? 10. What kind of support do you feel is lacking for middle-aged moms? How could this support make a difference in your life? End of survey Return your answers on one line, in a comma-separated list of your responses, with square brackets and each answer in quotes E.g., [""A"", ""B"", ""C""] After the answers, you can put a comment explaining your choice on the next line.","You are being asked to provide answers to a paper survey. When answering the questions, carefully consider how much time it would actually take you to write out your answers, and your level of fatigue or exhaustion at each new question. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.'}"


In [17]:
results = q.by(a).run()

In [18]:
results.select("multi", "multi_comment")

,answer.multi,comment.multi_comment
0,"['Balancing work and family time', 'I try to exercise regularly and meal prep on weekends', 'Dealing with teenage challenges', 'I shifted from full-time to part-time work', 'Using apps for scheduling and communication', 'Social life has become more family-oriented', 'Budgeting and using apps to track expenses', 'Reading and gardening; I fit them in during weekends', 'Improving work-life balance', 'More community support and resources for moms would help']",I kept my answers concise due to the time and effort it would take to write out detailed responses on a paper survey.


Comparing individual responses:

In [19]:
from edsl import QuestionFreeText, Survey

q1 = QuestionFreeText(
    question_name = "q1",
    question_text = "What is the most challenging aspect of your daily routine?"
)

q2 = QuestionFreeText(
    question_name = "q2",
    question_text = "How do you prioritize your health and wellness amidst your responsibilities? Please describe your strategies."
)

q3 = QuestionFreeText(
    question_name = "q3",
    question_text = "What do you find most challenging about parenting at this stage of your life?"
)

q4 = QuestionFreeText(
    question_name = "q4",
    question_text = "How has your career or educational pursuits changed since becoming a mother? Describe any shifts or turns you have experienced."
)

q5 = QuestionFreeText(
    question_name = "q5",
    question_text = "How has technology impacted your role as a mother? Please provide specific examples."
)

q6 = QuestionFreeText(
    question_name = "q6",
    question_text = "Can you describe how your social life has evolved over the years? What factors have influenced these changes?"
)

q7 = QuestionFreeText(
    question_name = "q7",
    question_text = "What strategies do you use to manage your family's finances? Please share any tips or challenges you've encountered."
)

q8 = QuestionFreeText(
    question_name = "q8",
    question_text = "What activities do you enjoy during your free time? How do you balance leisure with other responsibilities?"
)

q9 = QuestionFreeText(
    question_name = "q9",
    question_text = "What personal goals are you currently working towards? How do these goals reflect your life priorities?"
)

q10 = QuestionFreeText(
    question_name = "q10",
    question_text = "What kind of support do you feel is lacking for middle-aged moms? How could this support make a difference in your life?"
)

survey = Survey([q1,q2,q3,q4,q5,q6,q7,q8,q9,q10])

Applying full memory of prior questions:

In [20]:
full_memory = survey.set_full_memory_mode()

Inspecting the prompts:

In [21]:
full_memory.by(a).prompts().select("user_prompt", "system_prompt")

,user_prompt,system_prompt
0,What is the most challenging aspect of your daily routine?,"You are being asked to provide answers to a paper survey. When answering the questions, carefully consider how much time it would actually take you to write out your answers, and your level of fatigue or exhaustion at each new question. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.'}"
1,"How do you prioritize your health and wellness amidst your responsibilities? Please describe your strategies. Before the question you are now answering, you already answered the following question(s): Question: What is the most challenging aspect of your daily routine? Answer: None","You are being asked to provide answers to a paper survey. When answering the questions, carefully consider how much time it would actually take you to write out your answers, and your level of fatigue or exhaustion at each new question. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.'}"
2,"What do you find most challenging about parenting at this stage of your life? Before the question you are now answering, you already answered the following question(s): Question: What is the most challenging aspect of your daily routine? Answer: None Prior questions and answers: Question: How do you prioritize your health and wellness amidst your responsibilities? Please describe your strategies. Answer: None","You are being asked to provide answers to a paper survey. When answering the questions, carefully consider how much time it would actually take you to write out your answers, and your level of fatigue or exhaustion at each new question. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.'}"
3,"How has your career or educational pursuits changed since becoming a mother? Describe any shifts or turns you have experienced. Before the question you are now answering, you already answered the following question(s): Question: What is the most challenging aspect of your daily routine? Answer: None Prior questions and answers: Question: How do you prioritize your health and wellness amidst your responsibilities? Please describe your strategies. Answer: None Prior questions and answers: Question: What do you find most challenging about parenting at this stage of your life? Answer: None","You are being asked to provide answers to a paper survey. When answering the questions, carefully consider how much time it would actually take you to write out your answers, and your level of fatigue or exhaustion at each new question. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.'}"
4,"How has technology impacted your role as a mother? Please provide specific examples. Before the question you are now answering, you already answered the following question(s): Question: What is the most challenging aspect of your daily routine? Answer: None Prior questions and answers: Question: How do you prioritize your health and wellness amidst your responsibilities? Please describe your strategies. Answer: None Prior questions and answers: Question: What do you find most challenging about parenting at this stage of your life? Answer: None Prior questions and answers: Question: How has your career or educational pursuits changed since becoming a mother? Describe any shifts or turns you have experienced. Answer: None","You are being asked to provide answers to a paper survey. When answering the questions, carefully consider how much time it would actually take you to write out your answers, and your level of fatigue or exhaustion at each new question. Your traits: {'persona': 'You are a 45-year-old mom living in Massachusetts.'}"
5,"Can you describe how your social life has evolved over the years? What factors have influenced these changes? Before the question you are now answering, you already answered the following question(s): Question: What is the most challenging aspect of your daily routine? Answer: None P

In [22]:
results = full_memory.by(a).run()

In [23]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,agent.persona
4,answer.q1
5,answer.q10
6,answer.q2
7,answer.q3
8,answer.q4
9,answer.q5


In [24]:
results.select("q1", "q2", "q3", "q4", "q5", "q6", "q7", "q8", "q9", "q10")

,answer.q1,answer.q2,answer.q3,answer.q4,answer.q5,answer.q6,answer.q7,answer.q8,answer.q9,answer.q10
0,"Juggling work, kids' schedules, and finding time for myself.","I try to prioritize my health and wellness by setting small, achievable goals. I schedule regular exercise, like morning walks or yoga, which fit around my family's routine. I also meal plan on weekends to ensure we eat healthy throughout the week. I make sure to carve out a little time each day for myself, whether it's reading or a quick meditation session. Balancing everything is tough, but these strategies help me stay on track.","At this stage of my life, the most challenging aspect of parenting is managing the emotional and social needs of my kids as they grow older. They're becoming more independent, which is wonderful, but it also means navigating more complex issues like friendships, school pressures, and their growing desire for autonomy. Balancing being supportive while giving them space to grow is a delicate act. Additionally, keeping up with their technology use and ensuring they're safe online is an ongoing challenge.","Since becoming a mother, my career has definitely taken some turns. Before having kids, I was very focused on advancing in my field and took on roles that required a lot of time and travel. Once I became a mom, I shifted to positions that offered more flexibility, even if it meant slowing down my career progression. I also explored part-time work and remote opportunities to be more present for my children. Overall, my priorities shifted from climbing the career ladder to finding a balance that allows me to be there for my family while still pursuing my professional goals.","Technology has significantly impacted my role as a mother in several ways. For instance, it helps me stay organized and connected with my kids' schedules through calendar apps, which send reminders for school events, appointments, and extracurricular activities. Additionally, technology allows me to work from home more effectively, giving me the flexibility to be present for my children when they need me. Social media and messaging apps keep me connected with other parents and family members, offering a support network and a way to share parenting tips and experiences. However, managing screen time and ensuring my kids are safe online is a constant challenge, requiring me to stay informed and set boundaries around their technology use.","Over the years, my social life has evolved quite a bit, largely influenced by changes in my personal and family responsibilities. Before having kids, I had more freedom to spontaneously meet friends for dinner or engage in social activities. My social circle was broader, and I often participated in group events or hobbies. Once I became a mother, my social life shifted significantly. Much of my time and energy became focused on my children and their needs. Social interactions often revolved around playdates, school events, or activities related to my kids. I found myself connecting more with other parents who were in similar life stages, which was both comforting and practical. As my kids have grown older and more independent, I've started to reclaim some of my social time. I've reconnected with old friends and made a conscious effort to schedule regular catch-ups, whether it's a coffee date or a weekend outing. Technology has also played a role, making it easier to stay in touch with friends and family who live far away.","To manage my family's finances, I use a combination of budgeting, planning, and prioritizing our expenses. I start by setting a monthly budget that covers all necessary expenses like mortgage, utilities, groceries, and any recurring bills. I use budgeting apps to track our spending and ensure we stay on target. Meal planning helps reduce food waste and keeps grocery costs in check. We also set aside savings for emergencies and future goals, like college funds for the kids or family vacations. One challenge is b

### Example 4
In this example we prompt an agent to note salient features of textual content and images.

We start by posting some content and images to Coop so that they can be accessed by others:

In [25]:
from edsl import FileStore

fs = FileStore("waldo.png") # an image stored locally
info = fs.push(description = "Where's Waldo?", visibility = "public")
info

{'description': "Where's Waldo?",
 'object_type': 'scenario',
 'url': 'https://www.expectedparrot.com/content/ceda16ec-4146-4384-950e-6fb447c01619',
 'uuid': 'ceda16ec-4146-4384-950e-6fb447c01619',
 'version': '0.1.40.dev1',
 'visibility': 'public'}

Here we retrieve the file and use it in a scenario in order to add it to questions:

In [26]:
from edsl import Scenario

s = Scenario({"waldo":fs.pull(info["uuid"])})

We need to use a vision model with images, and verify that the model can actually "see" the images that we are using:

In [27]:
from edsl import Model, ModelList

m = ModelList(
    Model(model) for model in [
        "gemini-1.5-flash", 
        "gpt-4o"
    ]
)

In [28]:
from edsl import QuestionFreeText, QuestionList, Survey

q1 = QuestionFreeText(
    question_name = "salient",
    question_text = "Describe the most salient features of this image: {{ waldo }}"
)

q2 = QuestionList(
    question_name = "most_salient",
    question_text = "Describe the 3 most salient features of this image: {{ waldo }}"
)

q3 = QuestionFreeText(
    question_name = "where",
    question_text = "Where is Waldo in this picture: {{ waldo }}"
)

survey = Survey([q1, q2, q3])

In [29]:
results = survey.by(s).by(m).run()

In [30]:
results.select("model", "salient", "most_salient", "most_salient_comment", "where")

,model.model,answer.salient,answer.most_salient,comment.most_salient_comment,answer.where
0,gemini-1.5-flash,"Here's a description of the salient features of the image: The image is a highly detailed and densely populated illustration in the style of a ""Where's Waldo?"" puzzle. It depicts a large-scale scene of what appears to be a siege or battle in ancient times. The most prominent feature is a large stone fortress, situated centrally, which is surrounded by a vast multitude of people. These individuals are dressed in various styles of ancient Greek or Roman-inspired clothing and armor, suggesting warring factions. Some are on the walls of the fortress, others are attacking or defending below. The colors are vibrant and cartoonish, with distinct color-coding seemingly separating different groups of people.","['A large stone fortress under siege', 'Two opposing armies clashing on the beach', 'A volcanic eruption in the background']","The image depicts a scene of ancient warfare, with a central focus on a fortified structure under attack. The armies are clearly differentiated by their colors, and the volcanic eruption adds a dramatic element to the background.",Waldo is located near the bottom-center of the image. He's wearing his signature red and white striped shirt and is standing near a group of people with blue shields.
1,gpt-4o,"This image is a detailed and busy scene typical of a ""Where's Waldo?"" illustration. It features: 1. **Crowded Scene**: The image is densely packed with numerous characters and activities, making it visually complex. 2. **Medieval Setting**: There is a large stone fortress with towers and battlements, suggesting a historical or fantasy setting. 3. **Colorful Costumes**: Characters are dressed in various colorful outfits, primarily in red, blue, and yellow. 4. **Battle or Festival**: The scene resembles a battle or a large gathering, with groups of people holding shields and weapons. 5. **Water Features**: There are bodies of water on the sides, with boats and bridges indicating a coastal or river setting. 6. **Chaotic Activity**: The image is filled with chaotic and humorous interactions among the characters.","['Crowded scene with many people', 'Ancient city walls', 'Colorful outfits and shields']","The image features a densely populated scene with numerous characters, a prominent ancient city wall structure, and a variety of vibrant outfits and shields, creating a lively and intricate visual.","In the image, Waldo is located near the bottom right corner, just above the group of soldiers wearing yellow. Look for his distinctive red and white striped shirt and hat."


### Posting this notebook to Coop:

In [1]:
from edsl import Notebook

n = Notebook("agent_fatigue.ipynb")
info = n.push("Methods for exploring agent fatigue and memory", visibility = "public")
info

{'description': 'Methods for exploring agent fatigue and memory',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/1851a2bc-fb01-4594-9361-12a1a09d817b',
 'uuid': '1851a2bc-fb01-4594-9361-12a1a09d817b',
 'version': '0.1.40.dev1',
 'visibility': 'public'}

Updating an object at Coop:

In [32]:
from edsl import Notebook

n = Notebook("agent_fatigue.ipynb") # resave
n.patch(uuid = "1851a2bc-fb01-4594-9361-12a1a09d817b", value = n)

{'status': 'success'}